In [0]:
SET use_cached_result = false;
WITH inv AS 
(
    SELECT  w_warehouse_name
            ,w_warehouse_sk
            ,i_item_sk
            ,d_moy
            ,stdev
            ,mean
            ,CASE    mean
                    WHEN 0 THEN NULL
                    ELSE stdev / mean
            END cov
    FROM    (
                SELECT  w_warehouse_name
                        ,w_warehouse_sk
                        ,i_item_sk
                        ,d_moy
                        ,STDDEV_SAMP(inv_quantity_on_hand) stdev
                        ,AVG(inv_quantity_on_hand) mean
                FROM samples.tpcds_sf1000.inventory
                ,       samples.tpcds_sf1000.item
                ,       samples.tpcds_sf1000.warehouse
                ,       samples.tpcds_sf1000.date_dim
                WHERE   inv_item_sk = i_item_sk
                AND     inv_warehouse_sk = w_warehouse_sk
                AND     inv_date_sk = d_date_sk
                AND     d_year = 2000
                GROUP BY w_warehouse_name
                          ,w_warehouse_sk
                          ,i_item_sk
                          ,d_moy
            ) foo
    WHERE   CASE    mean
            WHEN 0 THEN 0
            ELSE stdev / mean
    END > 1
)
SELECT  inv1.w_warehouse_sk
        ,inv1.i_item_sk
        ,inv1.d_moy
        ,inv1.mean
        ,inv1.cov
        ,inv2.w_warehouse_sk
        ,inv2.i_item_sk
        ,inv2.d_moy
        ,inv2.mean
        ,inv2.cov
FROM inv inv1
,       inv inv2
WHERE   inv1.i_item_sk = inv2.i_item_sk
AND     inv1.w_warehouse_sk = inv2.w_warehouse_sk
AND     inv1.d_moy = 3
AND     inv2.d_moy = 3 + 1
ORDER BY inv1.w_warehouse_sk,inv1.i_item_sk,inv1.d_moy,inv1.mean,inv1.cov,inv2.d_moy,inv2.mean,inv2.cov
;